# IBM Data Science Professional Certification by Coursera

## Applied Data Science Capstone

## Capstone Project Week-5 - The Battle of Neighborhoods

## Starting a New Hospital for COVID-19 in Banglore, India

* Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page

* Get the geographical coordinates of the neighborhoods

* Obtain the venue data for the neighborhoods from Foursquare API

* Explore and cluster the neighborhoods

* Select the best cluster to start a New Hospital for COVID-19

## 1. Import Libraries 

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geocoder --yes
import requests # library to handle requests
import geocoder# library to handle requests

!conda install -c anaconda beautifulsoup4 --yes
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |           py36_0         168 KB  anaconda
    ca-certificates-2020.6.24  |                0         133 KB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

  be

## 2. Scrap data from Wikipedia page into a DataFrame

In [9]:
# send the GET request
data = requests.get("https://commons.wikimedia.org/wiki/Category:Suburbs_of_Bangalore").text

In [10]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [11]:
# create a list to store neighborhood data
neighborhoodList = []

In [12]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [13]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,"► Agara, Bangalore‎ (2 C, 6 F)"
1,► Arekere‎ (5 F)
2,"► Banashankari‎ (1 C, 5 F)"
3,► Banaswadi‎ (2 F)
4,"► Basavanagudi‎ (5 C, 11 F)"


In [14]:
kl_df.shape

(59, 1)

## 3. Get the geographical coordinates

In [15]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [19]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]
coords

[[12.841272755642173, 77.48159794529579],
 [12.997990005183027, 77.61046999022852],
 [12.939339984645756, 77.54449996982913],
 [12.997840011025701, 77.61036999412363],
 [12.938980000000072, 77.57137000000006],
 [12.882450000000063, 77.62475000000006],
 [12.927350000000047, 77.67185000000006],
 [12.981180014545643, 77.62507000827469],
 [12.966180000000065, 77.58690000000007],
 [12.957999987042513, 77.60094001777514],
 [12.817530000000033, 77.67879000000005],
 [12.966223851801814, 77.60679161806351],
 [12.85545000008095, 77.73125002986342],
 [12.987949993921418, 77.6065900072486],
 [12.99787001823907, 77.61040000133698],
 [12.943290000000047, 77.65602000000007],
 [12.839890330733185, 77.67220534445676],
 [12.998940000000061, 77.61276000000004],
 [12.942790000000059, 77.54122000000007],
 [13.047449981350198, 77.63327999861599],
 [13.049810000000036, 77.58903000000004],
 [12.957450008465656, 77.60091001056179],
 [12.912160000000029, 77.64490000000006],
 [12.978220000000022, 77.633970000000

In [20]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [21]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [22]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(59, 3)


,Neighborhood,Latitude,Longitude
0,"► Agara, Bangalore‎ (2 C, 6 F)",12.841273,77.481598
1,► Arekere‎ (5 F),12.997990,77.610470
2,"► Banashankari‎ (1 C, 5 F)",12.939340,77.544500
3,► Banaswadi‎ (2 F),12.997840,77.610370
4,"► Basavanagudi‎ (5 C, 11 F)",12.938980,77.571370
5,"► Begur, Bangalore‎ (1 C, 6 F)",12.882450,77.624750
6,"► Bellandur‎ (1 C, 4 F)",12.927350,77.671850
7,► BEML‎ (7 F),12.981180,77.625070
8,"► Bengaluru Pete‎ (9 C, 4 F)",12.966180,77.586900
9,"► Bidadi‎ (2 C, 2 F)",12.958000,77.600940


In [23]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

## 4. Create a map of Bangalore with neighborhoods superimposed on top

In [28]:
# get the coordinates of Bangalore
address = 'Bangalore, India'
geolocator = Nominatim(user_agent="Capstone Project - The Battle of Neighborhoods")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [29]:
# create map of Bangalore using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [30]:
# save the map as HTML file
map_kl.save('map_kl.html')

## 5. Use the Foursquare API to explore the neighborhoods

In [31]:
# define Foursquare Credentials and Version
CLIENT_ID = 'ETOY24XVRMCRU3JYDCRWCBXA32ZGFRPWRWENSDSIFO32JWBR' # your Foursquare ID
CLIENT_SECRET = 'FJDFE50YCIEVJT1IWAZ5XMXRPOCLLSODTIMBGKYWEVMMY2XB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ETOY24XVRMCRU3JYDCRWCBXA32ZGFRPWRWENSDSIFO32JWBR
CLIENT_SECRET:FJDFE50YCIEVJT1IWAZ5XMXRPOCLLSODTIMBGKYWEVMMY2XB


### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [45]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [46]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4005, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,► Arekere‎ (5 F),12.99799,77.61047,Shahi Kabab,12.995927,77.613525,BBQ Joint
1,► Arekere‎ (5 F),12.99799,77.61047,Naturals Icecream,12.996912,77.611268,Ice Cream Shop
2,► Arekere‎ (5 F),12.99799,77.61047,Mangalore Pearl - Seafood Restaurant,12.994472,77.615551,Seafood Restaurant
3,► Arekere‎ (5 F),12.99799,77.61047,Richies - Frazer Town,13.000930,77.613150,Indian Restaurant
4,► Arekere‎ (5 F),12.99799,77.61047,Lassi Shop Downtown,13.000869,77.613731,Juice Bar


### Let's check how many venues were returned for each neighorhood

In [47]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
► Arekere‎ (5 F),100,100,100,100,100,100
► BEML‎ (7 F),100,100,100,100,100,100
"► Banashankari‎ (1 C, 5 F)",25,25,25,25,25,25
► Banaswadi‎ (2 F),100,100,100,100,100,100
"► Basavanagudi‎ (5 C, 11 F)",100,100,100,100,100,100
"► Begur, Bangalore‎ (1 C, 6 F)",15,15,15,15,15,15
"► Bellandur‎ (1 C, 4 F)",85,85,85,85,85,85
"► Bengaluru Pete‎ (9 C, 4 F)",100,100,100,100,100,100
"► Bidadi‎ (2 C, 2 F)",100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [48]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 203 uniques categories.


In [49]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['BBQ Joint', 'Ice Cream Shop', 'Seafood Restaurant',
       'Indian Restaurant', 'Juice Bar', 'Park', 'Department Store',
       'Fast Food Restaurant', 'Pakistani Restaurant', 'Tea Room',
       'Pizza Place', 'Hyderabadi Restaurant', 'Snack Place', 'Bakery',
       'Pub', 'Café', 'Eastern European Restaurant', 'Shopping Mall',
       'Middle Eastern Restaurant', 'Steakhouse', 'Concert Hall',
       'Sandwich Place', 'Donut Shop', 'Coffee Shop',
       'Chinese Restaurant', 'Hotel', 'Wine Shop', 'Market',
       "Women's Store", 'Badminton Court', 'Asian Restaurant',
       'Clothing Store', 'South Indian Restaurant', 'Jewelry Store',
       'Music Venue', 'Bar', 'Mediterranean Restaurant', 'Trail',
       'Event Space', 'Pool Hall', 'Candy Store', 'Salad Place',
       'Breakfast Spot', 'Electronics Store',
       'Vegetarian / Vegan Restaurant', 'Miscellaneous Shop',
       'Travel & Transport', 'Gas Station', 'Metro Station',
       'Train Station'], dtype=object)

In [50]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## 6. Analyze Neighborhood

In [51]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(4005, 204)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,► Arekere‎ (5 F),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [52]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(56, 204)


,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Photography Studio,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,► Arekere‎ (5 F),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.020000,0.010000,0.00,0.030000,0.010000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.090000,0.010000,0.00,0.00,0.000000,0.000000,0.030000,0.00,0.030000,0.000000,0.020000,0.010000,0.000000,0.000,0.000000,0.00,0.00,0.00000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.030000,0.000000,0.010000,0.030000,0.00,0.240000,0.00,0.000000,0.000000,0.000000,0.00,0.01,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,

In [53]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

33

### Create a new DataFrame for Hospital data only

In [54]:
kl_hospital = kl_grouped[["Neighborhoods","Hospital"]]

In [55]:
kl_hospital.head()

,Neighborhoods,Hospital
0,► Arekere‎ (5 F),0.0
1,► BEML‎ (7 F),0.0
2,"► Banashankari‎ (1 C, 5 F)",0.0
3,► Banaswadi‎ (2 F),0.0
4,"► Basavanagudi‎ (5 C, 11 F)",0.0


## 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Bangalore into 5 clusters.

In [57]:
# set number of clusters
kclusters = 4

kl_clustering = kl_hospital.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return_n_iter=True)


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [58]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_hospital.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [59]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Hospital,Cluster Labels
0,► Arekere‎ (5 F),0.0,2
1,► BEML‎ (7 F),0.0,2
2,"► Banashankari‎ (1 C, 5 F)",0.0,2
3,► Banaswadi‎ (2 F),0.0,2
4,"► Basavanagudi‎ (5 C, 11 F)",0.0,2


In [60]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(56, 5)


,Neighborhood,Hospital,Cluster Labels,Latitude,Longitude
0,► Arekere‎ (5 F),0.0,2,12.99799,77.61047
1,► BEML‎ (7 F),0.0,2,12.98118,77.62507
2,"► Banashankari‎ (1 C, 5 F)",0.0,2,12.93934,77.54450
3,► Banaswadi‎ (2 F),0.0,2,12.99784,77.61037
4,"► Basavanagudi‎ (5 C, 11 F)",0.0,2,12.93898,77.57137


In [61]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(56, 5)


,Neighborhood,Hospital,Cluster Labels,Latitude,Longitude
13,"► Domlur‎ (5 C, 23 F)",0.021739,1,12.943290,77.656020
30,"► Koramangala‎ (1 C, 12 F)",0.000000,2,12.920040,77.625460
31,"► Krishnarajapura‎ (3 C, 3 F)",0.000000,2,13.000390,77.683680
32,► Kundalahalli‎ (96 F),0.000000,2,12.967520,77.715000
33,"► Madiwala‎ (1 C, 6 F)",0.000000,2,12.956610,77.613550
34,"► Magadi‎ (2 C, 10 F)",0.000000,2,12.988230,77.605830
35,► Mahadevapura‎ (2 C),0.000000,2,12.958000,77.600940
36,► Majestic (Bangalore)‎ (1 C),0.000000,2,12.957450,77.600910
37,"► Malleswaram‎ (4 C, 2 F)",0.000000,2,12.995000,77.573460
38,"► Marathahalli‎ (8 C, 1 P, 30 F)",0.000000,2,12.954660,77.707520


### Finally, let's visualize the resulting clusters

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [63]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 8. Examine Clusters

### Cluster 0

In [64]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Hospital,Cluster Labels,Latitude,Longitude


### Cluster 1

In [65]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Hospital,Cluster Labels,Latitude,Longitude
13,"► Domlur‎ (5 C, 23 F)",0.021739,1,12.94329,77.65602


### Cluster 2

In [66]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Hospital,Cluster Labels,Latitude,Longitude
30,"► Koramangala‎ (1 C, 12 F)",0.0,2,12.920040,77.625460
31,"► Krishnarajapura‎ (3 C, 3 F)",0.0,2,13.000390,77.683680
32,► Kundalahalli‎ (96 F),0.0,2,12.967520,77.715000
33,"► Madiwala‎ (1 C, 6 F)",0.0,2,12.956610,77.613550
34,"► Magadi‎ (2 C, 10 F)",0.0,2,12.988230,77.605830
35,► Mahadevapura‎ (2 C),0.0,2,12.958000,77.600940
36,► Majestic (Bangalore)‎ (1 C),0.0,2,12.957450,77.600910
37,"► Malleswaram‎ (4 C, 2 F)",0.0,2,12.995000,77.573460
38,"► Marathahalli‎ (8 C, 1 P, 30 F)",0.0,2,12.954660,77.707520
39,► Mathikere‎ (1 C),0.0,2,13.030330,77.559680


### Cluster 3

In [67]:
kl_merged.loc[kl_merged['Cluster Labels'] == 3]

,Neighborhood,Hospital,Cluster Labels,Latitude,Longitude


#### OBSERVATIONS:
Main focusing hospital arround the Bangalore city with the highest number in cluster 2 and less number in cluster 1, cluster 0 and cluster 3 has totally no hospitals in the neighborhoods. 

This represents a great opportunity and high potential areas to open new hospital as there is high scarcity of hospitals. 

Meanwhile, hospitals in cluster 2 are likely suffering from intense competition due to high concentration of shopping malls. 

From another perspective, this also shows that the high concentration of hospitals are mostly happened in the central area of the city, with the suburb area still have very few hospitals. 

Therefore, this project recommends Government to start new hospitals in neighborhoods in cluster 0 and cluster 3. 

Lastly, Government is advised to avoid neighborhoods in cluster 2 which already have high concentration of hospitals.